## US Agriculture Land use Analysis

__Import US Agriculture Data and analyze land use to determine the various factors affecting land use. This will be subsequently used to determine detractors and motivators for implementing soil policies to rejuvenate soil. #savesoil__

### Import Data

Data Source: https://www.ers.usda.gov/developer/data-apis/arms-data-api/

In [ ]:
!python3 -m pip install --upgrade pip

In [ ]:
#current working dir - /Users/renga/Documents/SpringBoard Data Engineering Course/agri_env
!pip install --ignore-installed -r requirements.txt

In [2]:
import requests
import pandas as pd
import json

In [ ]:
auth = json.load(open('api_key.json', 'rb')) 
token = auth["key"]
report_url = 'https://api.ers.usda.gov/data/arms/report?api_key='+token
variable_url ='https://api.ers.usda.gov/data/arms/variable?api_key='+token

In [ ]:
reports = requests.get(report_url).json()
report_names =[i['name'] for i in reports['data']]
report_names

In [ ]:
variables = requests.get(variable_url).json()
variable_df = pd.DataFrame(variables['data'])
variable_df.head()

In [ ]:
states = json.load(open('us_states.json','rb'))
state_codes = [i['abbreviation'].lower() for i in states['data']]
state_codes

## Sample Usage
survey_url = 'https://api.ers.usda.gov/data/arms/surveydata?api_key='+token

data_json = {
 "year": ['2018','2019','2020'],
 "state":['al', 'ak', 'as', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'dc', 'fm', 'fl', 'ga', 'gu', 'hi', 'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'mh', 'md', 'ma', 'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj', 'nm', 'ny', 'nc', 'nd', 'mp', 'oh', 'ok', 'or', 'pw', 'pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'ut', 'vt', 'vi', 'va', 'wa', 'wv', 'wi', 'wy'],
 "report": ['Farm Business Balance Sheet', 'Farm Business Income Statement','Farm Business Financial Ratios','Structural Characteristics','Farm Business Debt Repayment Capacity','Government Payments','Operator Household Income','Operator Household Balance Sheet'],
 "variable": ['incfin', 'fropmnum', 'lnreale', 'anequip', 'oped2', 'igovtn1', 'drmaxd75', 'ivalinv', 'iothfm', 'evlvpur', 'conspayt', 'drcnum75', 'lctot', 'evloth', 'atot', 'drintp', 'otherpayt', 'igcfioth', 'drmaxd10', 'acothr', 'incfi', 'nfdebt', 'igcfidp', 'conspay2', 'inmon', 'igovccp1', 'fnw', 'evmainr', 'lntot', 'mlb2', 'igcfig', 'igovtt', 'igovtg2', 'etot', 'ten2', 'totofi', 'evutil', 'lcterm', 'actot', 'hhasset', 'kdebt', 'hhdebt', 'incfioth', 'oped1', 'igcfimlb', 'eftot', 'fasst', 'efins', 'oped4', 'mlbt', 'incfig', 'lcpay', 'kountn', 'igcfin', 'anbreed', 'tothhi', 'iliv', 'vprodtot', 'ophr2', 'dshort', 'igcficcp', 'drmaxpay', 'efrent', 'evlabor', 'lnnreale', 'farmhhi', 'nfnw', 'breedingadj', 'opoc3', 'ophr1', 'acliv', 'eftaxes', 'evcwork', 'lcint', 'frroanum', 'hhnw', 'ten1', 'acprpins', 'kount', 'otherpay1', 'evtot', 'fratnum', 'evfertc', 'ten3', 'agi', 'dtot', 'fmhhtot', 'igovdpt', 'kntmlb', 'icrop', 'endepr', 'kntcnsv', 'kntoth', 'accrop', 'acinpt', 'evfeed', 'efint', 'acgrow', 'fdebt', 'acres', 'tacres', 'opoc1', 'enben', 'antot', 'evoth', 'aninvest', 'oped3', 'frroenum', 'igovt', 'drinc', 'nfasst', 'frwcnum', 'evfuelo', 'incficcp', 'incfimlb', 'frcrnum', 'anreale', 'igcfi', 'evseedp', 'drdcm', 'igcficnsv', 'infi', 'netw', 'opoc2', 'kntccp', 'incfidp', 'igovdp1', 'incficnsv', 'kountg', 'igovccpt', 'drcnum10', 'kntdp', 'frdanum', 'ophr3', 'freconum', 'froenum', 'ophr4', 'frdcnum']
}


survey_data = requests.post(survey_url,json=data_json).json()

survey_data['data']

In [ ]:
year = 2016
param = {}
param["state"] = state_codes
survey_data = []
while year < 2021:
    param["year"] = year
    for i in variables['data']:
        param["variable"] = i['id']
        param["report"] = i['report']
        temp_data = requests.post(survey_url,json=param).json()
        for i in temp_data['data']:
            survey_data.append(i)
        #print(survey_data)
    year += 1 
             
df=pd.DataFrame(survey_data)
df.head()

In [ ]:
len(df.index)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.columns[df.isna().any()].tolist()

In [ ]:
df.loc[:, df.isna().any()]

In [ ]:
df[df.columns[df.isnull().any()]].isnull().sum() * 100 / df.shape[0]

In [ ]:
df_non_null = df.drop(['variable_group','variable_group_id'],axis=1)

In [ ]:
df_non_null.fillna(0, inplace=True)

In [ ]:
df_non_null.columns[df_non_null.isna().any()].tolist()

In [ ]:
df_non_null.columns
#df_non_null.nunique()

In [ ]:
df_non_null['statistic'].unique()
#df_subset = df_non_null.drop_duplicates(subset=['unreliable_estimate'])
#df_subset

In [ ]:
sparse_columns = dict(df_non_null.nunique())
columns_to_drop = []
for i in sparse_columns.keys():
    if sparse_columns[i] <= 2:
        columns_to_drop.append(i)
columns_to_drop.append('report')
df_non_null.drop(columns_to_drop,axis=1,inplace=True)

In [ ]:
df_subset = df_non_null.drop_duplicates().reset_index(drop=True)
rowcount = len(df_subset.index)

In [ ]:
#!pip install openpyxl
df_subset.to_excel('us_farm_structure_finance.xlsx')

In [ ]:
data_json = [{df_subset.loc[i,'category_value']:df_subset.loc[i,'estimate'],
            'index':i
           } for i in range(0,rowcount-1)]
df_category_subset = pd.DataFrame(data_json)

In [63]:
df = pd.read_excel('us_farm_structure_finance.xlsx')
df.head()

,Unnamed: 0,year,state,category,category_value,variable_id,variable_name,variable_sequence,variable_level,variable_unit,variable_description,estimate,median,statistic,rse
0,0,2016,Texas,Operator Age,34 years or younger,kount,Farms,10,1,Number,Estimated number of farms.,3931.0,0.0,TOTAL,0.0
1,1,2016,Texas,Operator Age,55 to 64 years old,kount,Farms,10,1,Number,Estimated number of farms.,72132.0,0.0,TOTAL,0.0
2,2,2016,Texas,Farm Typology,Off-farm occupation farms (2011 to present),kount,Farms,10,1,Number,Estimated number of farms.,140734.0,0.0,TOTAL,0.0
3,3,2016,Texas,Collapsed Farm Typology,Intermediate farms,kount,Farms,10,1,Number,Estimated number of farms.,57341.0,0.0,TOTAL,0.0
4,4,2016,Texas,Production Specialty,General Cash Grains,kount,Farms,10,1,Number,Estimated number of farms.,1616.0,0.0,TOTAL,0.0


In [64]:
df["state"].unique()

array(['Texas', 'Washington', 'Wisconsin', 'Iowa', 'Kansas', 'Georgia',
       'Illinois', 'Minnesota', 'California', 'Florida', 'Arkansas',
       'North Carolina', 'Indiana', 'Missouri', 'Nebraska'], dtype=object)

Determined that data exists pertaining to only 15 states. Also the value is questionable for some of the metrics. Hence, moving to a different dataset from USDA website which seems to have information across all states - CENSUS and SURVEY data

Requirement is to fetch following information as part of data exploration phase:
1. Determine total acres of crop land in the US across the 50 states and how much of it use organic production practices.
2. Determine states investing in organic production practice conversion and increase in organic farms from year to year
3. Determine representative value of organic matter present in soil and the percentiles (5%, 10%, average, 90% 95%) across the states in the US (in a separate Jupyter notebook - us_soil_data_analysis.ipynb)

## US Agriculture Quick Stats Data

Source: https://quickstats.nass.usda.gov/

Sample API: GET http://quickstats.nass.usda.gov/api/api_GET/?key={{QS_API_KEY}}&year__GE=2020&state_alpha=CA

This data will be used to satisfy requirement #1 and #2 above.

In [9]:
auth = json.load(open('qs_api_key.json', 'rb')) 
token = auth["key"]
url = 'http://quickstats.nass.usda.gov/api/api_GET/?key='+token

In [ ]:
state_codes = [i['abbreviation'] for i in states['data']]
state_codes

In [ ]:
year ='2016'
agri_data_url = url + '&year=' + str(year) + '&state_alpha=' + 'CA'
temp_data = requests.get(agri_data_url).json()
type(temp_data)
if 'error' in temp_data.keys():
    print(temp_data['error'][0])
else:
    print(temp_data['data'][0])

In [ ]:
year ='2016'
agri_data_url = url + '&year=' + str(year) + '&state_alpha=' + 'AL' + '&format=csv'
df = pd.read_csv(agri_data_url)
df.head()

In [ ]:
#json format gave error for CA state as json response from request was not complete. 
# hence not continuing in json format
year = 2016
agri_data = []
error_data = []
while year <= 2016:
    for i in state_codes:
        agri_data_url = url + '&year=' + str(year) + '&state_alpha=' + i
        print(agri_data_url)
        temp_data = requests.get(agri_data_url).json()
        try:
            if 'data' in temp_data.keys():
                for i in temp_data['data']:
                    agri_data.append(i)
            elif 'error' in temp_data.keys():
                error_data.append({"State":i,"Error":temp_data['error'][0]})
        except Exception as e:
            error_data.append({"State":i,"Error":e})
    year += 1 

    df=pd.DataFrame(agri_data)
    df_error = pd.DataFrame(error_data)
    df.head()

In [ ]:
##fetching data in csv format for specific units based on requirement
year = 2016
error_data = []
df_superset = pd.DataFrame()
units=['$','ACRES','GALLONS','LB']
while year <= 2018:
    for i in state_codes:
        for j in units:
            agri_data_url = url + '&year=' + str(year) + '&state_alpha=' + i + '&freq_desc=ANNUAL&reference_period_desc=YEAR&unit_desc='+j+'&format=csv' 
            print(agri_data_url)
            try:
                df = pd.read_csv(agri_data_url,low_memory=False) 
                df_superset = pd.concat([df_superset,df])
            except Exception as e:
                r = requests.get(agri_data_url)
                if r.status_code != 200:
                    error_data.append({"State":i,"Year":year,"Error":e})    
    year += 1 
df_error = pd.DataFrame(error_data)
df_superset.head()

In [ ]:
len(df_superset.index)
df_error

In [ ]:
year = 2019
error_data = []
df_set_2 = pd.DataFrame()
units=['$','ACRES','GALLONS','LB']
while year <= 2022:
    for i in state_codes:
        for j in units:
            agri_data_url = url + '&year=' + str(year) + '&state_alpha=' + i + '&freq_desc=ANNUAL&reference_period_desc=YEAR&unit_desc='+j+'&format=csv' 
            print(agri_data_url)
            try:
                df = pd.read_csv(agri_data_url,low_memory=False) 
                df_set_2 = pd.concat([df_set_2,df])
            except Exception as e:
                r = requests.get(agri_data_url)
                if r.status_code != 200:
                    error_data.append({"State":i,"Year":year,"Error":e})    
    year += 1 
df_error1 = pd.DataFrame(error_data)
df_set_2.head()

In [ ]:
#Obtaining what API fetch returned errors so can be handled separately. Errors 
#determined were owing to exceeding 50000 data limit, worked around by fetching only state level data
df_error = pd.concat([df_error,df_error1])

In [ ]:
df_superset = pd.concat([df_superset,df_set_2])
len(df_superset.index)

In [ ]:
df_superset.to_csv('us_agri_data_analysis.csv')

In [ ]:
df_error.to_excel('error.xlsx')

In [3]:
df_restore = pd.read_csv('us_agri_data_analysis.csv', low_memory=False)

## Analysis of Data from USDA QuickStats

In [21]:
len(df_restore.index)

2135869

In [46]:
df[df['state_alpha']=='TX' ]

,source_desc,sector_desc,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,statisticcat_desc,unit_desc,domain_desc,domaincat_desc,state_alpha,state_name,country_name,year,Values
37771,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,ANIMALS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),TX,TEXAS,UNITED STATES,2016,19797000.0
37772,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,PRODUCTS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),TX,TEXAS,UNITED STATES,2016,204436000.0
37773,CENSUS,ANIMALS & PRODUCTS,DAIRY,MILK,ALL CLASSES,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),TX,TEXAS,UNITED STATES,2016,164492000.0
37774,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,(EXCL COWS),ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),TX,TEXAS,UNITED STATES,2016,16754357.0
37775,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,"COWS, MILK",ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),TX,TEXAS,UNITED STATES,2016,2286867.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8598,SURVEY,ECONOMICS,FARMS & LAND & ASSETS,FARM OPERATIONS,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,AREA OPERATED,ACRES,ECONOMIC CLASS,"ECONOMIC CLASS: (10,000 TO 99,999 $)",TX,TEXAS,UNITED STATES,2017,39100000.0
8599,SURVEY,ECONOMICS,FARMS & LAND & ASSETS,FARM OPERATIONS,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,AREA OPERATED,ACRES,ECONOMIC CLASS,"ECONOMIC CLASS: (100,000 TO 249,999 $)",TX,TEXAS,UNITED STATES,2017,19200000.0
8600,SURVEY,ECONOMICS,FARMS & LAND & ASSETS,FARM OPERATIONS,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,AREA OPERATED,ACRES,ECONOMIC CLASS,"ECONOMIC CLASS: (250,000 TO 499,999 $)",TX,TEXAS,UNITED STATES,2017,15200000.0
8601,SURVEY,ECONOMICS,FARMS & LAND & ASSETS,FARM OPERATIONS,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,AREA OPERATED,ACRES,ECONOMIC CLASS,"ECONOMIC CLASS: (500,000 TO 999,999 $)",TX,TEXAS,UNITED STATES,2017,14300000.0


In [17]:
#Texas data fetched distinctly only for aggregate level state to avoid excess data fetch error
year = 2017
state_code ='TX'
units = 'ACRES'
error_data =[]
agri_data_url = url + '&year=' + str(year) + '&state_alpha=' + state_code + '&freq_desc=ANNUAL&reference_period_desc=YEAR&unit_desc='+units+'&agg_level_desc=STATE'+'&format=csv' 
print(agri_data_url)
try:
    df1 = pd.read_csv(agri_data_url,low_memory=False) 
    df = pd.concat([df,df1])
    del df1
except Exception as e:
    r = requests.get(agri_data_url)
    if r.status_code != 200:
        error_data.append({"State":state_code,"Year":year,"Error":e})     
df_error = pd.DataFrame(error_data)
df.head()

http://quickstats.nass.usda.gov/api/api_GET/?key=B9F99B15-6E68-36F3-A4CF-E66FD7DEA41D&year=2017&state_alpha=TX&freq_desc=ANNUAL&reference_period_desc=YEAR&unit_desc=ACRES&agg_level_desc=STATE&format=csv


,source_desc,sector_desc,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,statisticcat_desc,unit_desc,short_desc,...,location_desc,year,freq_desc,begin_code,end_code,reference_period_desc,week_ending,load_time,Value,CV (%)
0,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,INCL PRODUCTS,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SALES,$,"ANIMAL TOTALS, INCL PRODUCTS - SALES, MEASURED...",...,TEXAS,2017,ANNUAL,0,0,YEAR,NaN,2018-02-01 00:00:00,"4,955,921,000",2.2
1,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,INCL PRODUCTS,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SALES,$,"ANIMAL TOTALS, INCL PRODUCTS - SALES, MEASURED...",...,TEXAS,2017,ANNUAL,0,0,YEAR,NaN,2018-02-01 00:00:00,"2,614,598,000",2.6
2,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,INCL PRODUCTS,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SALES,$,"ANIMAL TOTALS, INCL PRODUCTS - SALES, MEASURED...",...,TEXAS,2017,ANNUAL,0,0,YEAR,NaN,2018-02-01 00:00:00,"13,073,814,000",2.2
3,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,INCL PRODUCTS,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SALES,$,"ANIMAL TOTALS, INCL PRODUCTS - SALES, MEASURED...",...,TEXAS,2017,ANNUAL,0,0,YEAR,NaN,2018-02-01 00:00:00,"288,694,000",4.8
4,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,INCL PRODUCTS,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,SALES,$,"ANIMAL TOTALS, INCL PRODUCTS - SALES, MEASURED...",...,TEXAS,2017,ANNUAL,0,0,YEAR,NaN,2018-02-01 00:00:00,"100,710,000",6.2


In [18]:
len(df.index)

49007

In [20]:
df_restore = pd.concat([df_restore,df])

In [22]:
df_restore.drop(['Unnamed: 0'],axis = 1,inplace = True)


In [23]:
df_restore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2135869 entries, 0 to 8602
Data columns (total 39 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   source_desc            object 
 1   sector_desc            object 
 2   group_desc             object 
 3   commodity_desc         object 
 4   class_desc             object 
 5   prodn_practice_desc    object 
 6   util_practice_desc     object 
 7   statisticcat_desc      object 
 8   unit_desc              object 
 9   short_desc             object 
 10  domain_desc            object 
 11  domaincat_desc         object 
 12  agg_level_desc         object 
 13  state_ansi             int64  
 14  state_fips_code        int64  
 15  state_alpha            object 
 16  state_name             object 
 17  asd_code               float64
 18  asd_desc               object 
 19  county_ansi            float64
 20  county_code            float64
 21  county_name            object 
 22  region_desc          

In [25]:
df_restore.nunique()

source_desc                   2
sector_desc                   5
group_desc                   17
commodity_desc              317
class_desc                  986
prodn_practice_desc          82
util_practice_desc          107
statisticcat_desc            37
unit_desc                     4
short_desc                 6516
domain_desc                 160
domaincat_desc             2683
agg_level_desc                5
state_ansi                   55
state_fips_code              55
state_alpha                  55
state_name                   55
asd_code                     25
asd_desc                     88
county_ansi                 267
county_code                 274
county_name                1789
region_desc                  41
zip_5                         0
watershed_code                1
watershed_desc                0
congr_district_code           0
country_code                  6
country_name                  6
location_desc              3830
year                          7
freq_des

In [26]:
df = df_restore.drop(['short_desc','state_ansi',
'state_fips_code','asd_code','county_ansi','county_code','zip_5','watershed_code','watershed_desc',
'congr_district_code','location_desc','freq_desc','begin_code','end_code','reference_period_desc',
'week_ending','load_time','CV (%)'],axis=1)

In [27]:
df.head()

,source_desc,sector_desc,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,statisticcat_desc,unit_desc,domain_desc,...,agg_level_desc,state_alpha,state_name,asd_desc,county_name,region_desc,country_code,country_name,year,Value
0,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,ANIMALS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,STATE,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,(D)
1,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,PRODUCTS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,STATE,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,(D)
2,CENSUS,ANIMALS & PRODUCTS,DAIRY,MILK,ALL CLASSES,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,STATE,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,(D)
3,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,(EXCL COWS),ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,STATE,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,(D)
4,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,"COWS, BEEF",ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,STATE,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,(D)


In [28]:
df['Values'] = df['Value'].astype('str')

In [29]:
import numpy as np
df[df['Values'].str.contains('\(')]

,source_desc,sector_desc,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,statisticcat_desc,unit_desc,domain_desc,...,state_alpha,state_name,asd_desc,county_name,region_desc,country_code,country_name,year,Value,Values
0,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,ANIMALS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,(D),(D)
1,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,PRODUCTS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,(D),(D)
2,CENSUS,ANIMALS & PRODUCTS,DAIRY,MILK,ALL CLASSES,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,(D),(D)
3,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,(EXCL COWS),ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,(D),(D)
4,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,"COWS, BEEF",ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,(D),(D)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8436,CENSUS,ENVIRONMENTAL,FARMS & LAND & ASSETS,AG LAND,ALL CLASSES,ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,TREATED,ACRES,"NAICS CLASSIFICATION AND CHEMICAL, OTHER",...,TX,TEXAS,NaN,NaN,NaN,9000,UNITED STATES,2017,(D),(D)
8489,SURVEY,CROPS,FIELD CROPS,BEANS,"DRY EDIBLE, OTHER, NOT LISTED",ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,AREA HARVESTED,ACRES,TOTAL,...,TX,TEXAS,NaN,NaN,NaN,9000,UNITED STATES,2017,(D),(D)
8490,SURVEY,CROPS,FIELD CROPS,BEANS,"DRY EDIBLE, OTHER, NOT LISTED",ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,AREA PLANTED,ACRES,TOTAL,...,TX,TEXAS,NaN,NaN,NaN,9000,UNITED STATES,2017,(D),(D)
8491,SURVEY,CROPS,FIELD CROPS,BEANS,"DRY EDIBLE, PINTO",ALL PRODUCTION PRACTICES,ALL UTILIZATION PRACTICES,AREA HARVESTED,ACRES,TOTAL,...,TX,TEXAS,NaN,NaN,NaN,9000,UNITED STATES,2017,(D),(D)


In [30]:
##Converting values to float value representing the amount of acres
df['Values'] = df['Values'].str.replace(',','')
df['Values'] = np.where(df['Values'].str.contains('\('),0,df['Values'])
df['Values'] = df['Values'].astype('float')
df.drop(['Value'],axis = 1,inplace = True)
df.head()

,source_desc,sector_desc,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,statisticcat_desc,unit_desc,domain_desc,...,agg_level_desc,state_alpha,state_name,asd_desc,county_name,region_desc,country_code,country_name,year,Values
0,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,ANIMALS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,STATE,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,0.0
1,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,PRODUCTS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,STATE,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,0.0
2,CENSUS,ANIMALS & PRODUCTS,DAIRY,MILK,ALL CLASSES,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,STATE,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,0.0
3,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,(EXCL COWS),ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,STATE,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,0.0
4,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,"COWS, BEEF",ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,...,STATE,AL,ALABAMA,NaN,NaN,NaN,9000,UNITED STATES,2016,0.0


In [31]:
df.drop(['asd_desc','county_name','region_desc'],axis = 1,inplace = True)
df.head()

,source_desc,sector_desc,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,statisticcat_desc,unit_desc,domain_desc,domaincat_desc,agg_level_desc,state_alpha,state_name,country_code,country_name,year,Values
0,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,ANIMALS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),STATE,AL,ALABAMA,9000,UNITED STATES,2016,0.0
1,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,PRODUCTS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),STATE,AL,ALABAMA,9000,UNITED STATES,2016,0.0
2,CENSUS,ANIMALS & PRODUCTS,DAIRY,MILK,ALL CLASSES,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),STATE,AL,ALABAMA,9000,UNITED STATES,2016,0.0
3,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,(EXCL COWS),ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),STATE,AL,ALABAMA,9000,UNITED STATES,2016,0.0
4,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,"COWS, BEEF",ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),STATE,AL,ALABAMA,9000,UNITED STATES,2016,0.0


In [ ]:
a = df['Values'].unique()
print(sorted(a))
##' (D)', ' (NA)', ' (X)', ' (Z)', '(1)', '(O)',

In [96]:
#df['country_name'].unique()
df['sector_desc'].unique()

array(['ANIMALS & PRODUCTS', 'CROPS', 'ECONOMICS', 'ENVIRONMENTAL',
       'DEMOGRAPHICS'], dtype=object)

In [32]:
df['year'].unique()

array([2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [33]:
df = df[df["agg_level_desc"]=='STATE']

In [34]:
df.head()

,source_desc,sector_desc,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,statisticcat_desc,unit_desc,domain_desc,domaincat_desc,agg_level_desc,state_alpha,state_name,country_code,country_name,year,Values
0,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,ANIMALS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),STATE,AL,ALABAMA,9000,UNITED STATES,2016,0.0
1,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,PRODUCTS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),STATE,AL,ALABAMA,9000,UNITED STATES,2016,0.0
2,CENSUS,ANIMALS & PRODUCTS,DAIRY,MILK,ALL CLASSES,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),STATE,AL,ALABAMA,9000,UNITED STATES,2016,0.0
3,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,(EXCL COWS),ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),STATE,AL,ALABAMA,9000,UNITED STATES,2016,0.0
4,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,"COWS, BEEF",ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),STATE,AL,ALABAMA,9000,UNITED STATES,2016,0.0


In [ ]:
df.drop(['agg_level_desc','country_code'],axis = 1,inplace = True)

In [39]:
len(df.index)

924380

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 908365 entries, 0 to 2086861
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   source_desc          908365 non-null  object 
 1   sector_desc          908365 non-null  object 
 2   group_desc           908365 non-null  object 
 3   commodity_desc       908365 non-null  object 
 4   class_desc           908365 non-null  object 
 5   prodn_practice_desc  908365 non-null  object 
 6   util_practice_desc   908365 non-null  object 
 7   statisticcat_desc    908365 non-null  object 
 8   unit_desc            908365 non-null  object 
 9   domain_desc          908365 non-null  object 
 10  domaincat_desc       908365 non-null  object 
 11  state_alpha          908365 non-null  object 
 12  state_name           908365 non-null  object 
 13  country_name         908365 non-null  object 
 14  year                 908365 non-null  int64  
 15  Values          

In [41]:
df_groupby = df.groupby(['year','country_name','state_name','state_alpha','source_desc','statisticcat_desc','unit_desc',
'sector_desc','group_desc','commodity_desc','class_desc','domain_desc','domaincat_desc','prodn_practice_desc',
'util_practice_desc']).sum()

In [42]:
df_groupby.to_csv('us_agri_group_by.csv')

In [50]:
df.head()

,source_desc,sector_desc,group_desc,commodity_desc,class_desc,prodn_practice_desc,util_practice_desc,statisticcat_desc,unit_desc,domain_desc,domaincat_desc,state_alpha,state_name,country_name,year,Values
0,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,ANIMALS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),AL,ALABAMA,UNITED STATES,2016,0.0
1,CENSUS,ANIMALS & PRODUCTS,ANIMAL TOTALS,ANIMAL TOTALS,PRODUCTS ONLY,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),AL,ALABAMA,UNITED STATES,2016,0.0
2,CENSUS,ANIMALS & PRODUCTS,DAIRY,MILK,ALL CLASSES,ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),AL,ALABAMA,UNITED STATES,2016,0.0
3,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,(EXCL COWS),ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),AL,ALABAMA,UNITED STATES,2016,0.0
4,CENSUS,ANIMALS & PRODUCTS,LIVESTOCK,CATTLE,"COWS, BEEF",ORGANIC,ALL UTILIZATION PRACTICES,SALES,$,ORGANIC STATUS,ORGANIC STATUS: (NOP USDA CERTIFIED),AL,ALABAMA,UNITED STATES,2016,0.0


In [54]:
##Filter data for requirement #1 and #2
df_area = df[(df['statisticcat_desc']=='AREA') & (df['commodity_desc']=='AG LAND')
& (df['sector_desc']=='ECONOMICS') & (df['class_desc']=='CROPLAND') 
& (df['prodn_practice_desc']!='IRRIGATED') & (df.domain_desc.isin(['ORGANIC STATUS','TOTAL']))]

In [66]:
#Export to csv and visualize in Tableau
df_area.to_csv("us_agri_data_2016_2022_explore.csv")

### Visualization in Tableau Dashboard
https://public.tableau.com/views/OrganicvsAllProductionPracticesAreainAcresbyUSState/OrganicvsAllProductionPracticeFarms?:language=en-US&:display_count=n&:origin=viz_share_link